In [30]:
from pydantic import BaseModel
import requests
import json
import tomllib as tl

with open("config.toml", "rb") as f:
    config = tl.load(f)
    ssm = config["SSMERP"]

class Service:
    base_url: str = "https://api.myinvois.hasil.gov.my"

class LoginSchema(BaseModel):
    client_id: str
    client_secret: str
    grant_type: str = "client_credentials"
    scope: str = "InvoicingAPI"

class LoginResponse(BaseModel):
    access_token: str
    token_type: str
    expires_in: int
    scope: str

class LoginService(Service):
    @classmethod
    def login(self, body: LoginSchema) -> LoginResponse:
        headers = {
            "Content-Type": "application/x-www-form-urlencoded"
        }
        res = requests.post(self.base_url + "/connect/token", data=body.model_dump(), headers=headers)
        return LoginResponse(
            **json.loads(res.text)
        )

body = LoginSchema(
    client_id=ssm["ClientID"],
    client_secret=ssm["Secret1"],
)
token = LoginService.login(body)
token.model_dump()

{'access_token': 'eyJhbGciOiJSUzI1NiIsImtpZCI6Ijk2RjNBNjU2OEFEQzY0MzZDNjVBNDg1MUQ5REM0NTlFQTlCM0I1NTRSUzI1NiIsIng1dCI6Imx2T21Wb3JjWkRiR1draFIyZHhGbnFtenRWUSIsInR5cCI6ImF0K2p3dCJ9.eyJpc3MiOiJodHRwczovL2lkZW50aXR5Lm15aW52b2lzLmhhc2lsLmdvdi5teSIsIm5iZiI6MTc0MjI3ODcwNSwiaWF0IjoxNzQyMjc4NzA1LCJleHAiOjE3NDIyODIzMDUsImF1ZCI6Imh0dHBzOi8vaWRlbnRpdHkubXlpbnZvaXMuaGFzaWwuZ292Lm15L3Jlc291cmNlcyIsInNjb3BlIjpbIkludm9pY2luZ0FQSSJdLCJjbGllbnRfaWQiOiI2OTEyMTM5Ni0yNDA3LTQxMDItYWZjYy1iZTdmNGVmOTAxMWUiLCJJc1RheFJlcHJlcyI6IjEiLCJJc0ludGVybWVkaWFyeSI6IjAiLCJJbnRlcm1lZElkIjoiMCIsIkludGVybWVkVElOIjoiIiwiSW50ZXJtZWRST0IiOiIiLCJJbnRlcm1lZEVuZm9yY2VkIjoiMiIsIm5hbWUiOiJJRzUwNTk4MjY2MDgwOjY5MTIxMzk2LTI0MDctNDEwMi1hZmNjLWJlN2Y0ZWY5MDExZSIsIlNTSWQiOiJlMDZmNjZhYS1iMjI4LTE4MzctZTFiMS0yZDhhYTdmMWZlYTIiLCJwcmVmZXJyZWRfdXNlcm5hbWUiOiJEZXYiLCJUYXhJZCI6IjE1NDExMSIsIlRheHBheWVyVElOIjoiSUc1MDU5ODI2NjA4MCIsIlByb2ZJZCI6IjE1ODU0MiIsIklzVGF4QWRtaW4iOiIwIiwiSXNTeXN0ZW0iOiIxIn0.WI2NSSR5gNPSyxtP_c9_Tld2zDql5y8Ngn6-ESls5LZeql__4PIAd

In [ ]:
url = "/api/v1.0/documenttypes"
headers = {
    "Authorization": f"{token.token_type} {token.access_token}"
}

res = requests.get(Service.base_url + url, headers=headers)
import pprint
pprint.pprint(json.loads(res.text))

In [33]:
url = "/api/v1.0/documenttypes/1"
headers = {
    "Authorization": f"{token.token_type} {token.access_token}"
}

res = requests.get(Service.base_url + url, headers=headers)
import pprint
pprint.pprint(json.loads(res.text))

{'activeFrom': '2024-05-02T10:34:11.15Z',
 'activeTo': None,
 'description': 'Invoice',
 'documentTypeVersions': [{'activeFrom': '2024-04-26T17:47:49.2Z',
                           'activeTo': None,
                           'description': 'Version 1',
                           'id': 1,
                           'name': 'Version 1',
                           'status': 'Published',
                           'versionNumber': '1.0'},
                          {'activeFrom': '2024-06-27T16:12:09.6Z',
                           'activeTo': None,
                           'description': 'Version 2',
                           'id': 2,
                           'name': 'Version 2',
                           'status': 'Published',
                           'versionNumber': '1.1'}],
 'id': 1,
 'invoiceTypeCode': '01',
 'workflowParameters': [{'activeFrom': '2024-03-14T04:59:24.076Z',
                         'activeTo': None,
                         'id': 1,
                         

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class Value(BaseModel):
    value: str = Field(..., alias="_")

class Item(BaseModel):
    description: str
    quantity: int
    unit_price: float

class Invoice(Value):
    ...
    items: List[Item]


invoice = Invoice(
    _="Test",
    items=[
        Item(
            description="Test",
            quantity=1,
            unit_price=10.00
        )
    ]
)
# invoice.model_dump()
Value(_="Hello").model_dump(by_alias=True)

{'_': 'Hello'}

In [8]:
from pydantic import BaseModel, Field
from typing import List, Any
import datetime

class ValueStr_(BaseModel):
    value: str = Field(..., alias="_")
class ValueInt_(BaseModel):
    value: int = Field(..., alias="_")
class ValueFloat_(BaseModel):
    value: float = Field(..., alias="_")
class ValueBool_(BaseModel):
    value: bool = Field(..., alias="_")
class ValueDate_(BaseModel):
    value: datetime.date = Field(..., alias="_")
class ValueDateTime_(BaseModel):
    value: datetime.datetime = Field(..., alias="_")
class ValueTime_(BaseModel):
    value: datetime.time = Field(..., alias="_")


class InvoiceTypeCode_(BaseModel):
    value: str = Field(default="01", alias="_")
    listVersionID: str = "1.0"
class DocumentCurrencyCode_(BaseModel):
    value: str = Field(default="MYR", alias="_")
class TaxCurrencyCode_(BaseModel):
    value: str = Field(default="MYR", alias="_")
class InvoicePeriod_(BaseModel):
    StartDate: List[ValueDate_]
    EndDate: List[ValueDate_]
    Description: List[ValueStr_]

class Invoice(BaseModel):
    ID: List[ValueStr_]
    IssueDate: List[ValueDate_]
    IssueTime: List[ValueTime_]
    InvoiceTypeCode: List[InvoiceTypeCode_]
    DocumentCurrencyCode: List[DocumentCurrencyCode_]
    TaxCurrencyCode: List[TaxCurrencyCode_]
    InvoicePeriod: List[InvoicePeriod_]

Invoice(
    **{
        "ID": [{"_": "INV001"}],
        "IssueDate": [{"_": "2021-01-01"}],
        "IssueTime": [{"_": "12:00:00"}],
        "InvoiceTypeCode": [{"_": "01"}],
        "DocumentCurrencyCode": [{"_": "MYR"}],
        "TaxCurrencyCode": [{"_": "MYR"}],
        "InvoicePeriod": [{
            "StartDate": [{"_": "2021-01-01"}],
            "EndDate": [{"_": "2021-01-31"}],
            "Description": [{"_": "January 2021"}]
        }]
    }
).model_dump(by_alias=True, mode="json")

{'ID': [{'_': 'INV001'}],
 'IssueDate': [{'_': '2021-01-01'}],
 'IssueTime': [{'_': '12:00:00'}],
 'InvoiceTypeCode': [{'_': '01', 'listVersionID': '1.0'}],
 'DocumentCurrencyCode': [{'_': 'MYR'}],
 'TaxCurrencyCode': [{'_': 'MYR'}],
 'InvoicePeriod': [{'StartDate': [{'_': '2021-01-01'}],
   'EndDate': [{'_': '2021-01-31'}],
   'Description': [{'_': 'January 2021'}]}]}

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Any
import datetime

class ValueStr_(BaseModel):
    value: str = Field(..., alias="_")
class ValueInt_(BaseModel):
    value: int = Field(..., alias="_")
class ValueFloat_(BaseModel):
    value: float = Field(..., alias="_")
class ValueBool_(BaseModel):
    value: bool = Field(..., alias="_")
class ValueDate_(BaseModel):
    value: datetime.date = Field(..., alias="_")
class ValueDateTime_(BaseModel):
    value: datetime.datetime = Field(..., alias="_")
class ValueTime_(BaseModel):
    value: datetime.time = Field(..., alias="_")


class AdditionalAccountID_(ValueStr_):
    schemeAgencyName: str = "CertEX"
class IndustryClassificationCode_(ValueStr_):
    name: str
class PartyIdentificationDetails_(ValueStr_):
    schemeID: str
class PartyIdentification_(BaseModel):
    ID: List[PartyIdentificationDetails_]
class AddressLine_(BaseModel):
    Line: List[ValueStr_]
class IdentificationCode_(ValueStr_):
    value: str = Field("MYS", alias="_")
    listID: str = "ISO3166-1"
    listAgencyID: str = "6"
class Country_(BaseModel):
    IdentificationCode: List[IdentificationCode_]

class PostalAddress_(BaseModel):
    CityName: List[ValueStr_]
    PostalZone: List[ValueStr_]
    CountrySubentityCode: List[ValueStr_]
    AddressLine: List[AddressLine_]
    Country: List[Country_]

'''
 "PartyLegalEntity": [
                {
                  "RegistrationName": [
                    {
                      "_": "Supplier's Name"
                    }
                  ]
                }
              ],
              "Contact": [
                {
                  "Telephone": [
                    {
                      "_": "+60-123456789"
                    }
                  ],
                  "ElectronicMail": [
                    {
                      "_": "supplier@email.com"
                    }
                  ]
                }
'''

class PartyLegalEntity_(BaseModel):
    RegistrationName: List[ValueStr_] = Field(..., alias="RegistrationName")
class Telephone_(BaseModel):
    value: str = Field(..., alias="_")
class ElectronicMail_(BaseModel):
    value: str = Field(..., alias="_")
class Contact_(BaseModel):
    Telephone: List[Telephone_]
    ElectronicMail: List[ElectronicMail_] = Field(..., alias="ElectronicMail")


class AccountingSupplierParty_(BaseModel):
    AdditionalAccountID: List[AdditionalAccountID_]
    PostalAddress: List[PostalAddress_]
    PartyIdentification: List[PartyIdentification_]
    PartyLegalEntity: List[PartyLegalEntity_]
    Contact: List[Contact_]


PostalAddress_(
    **{
        "CityName": [{"_": "Kuala Lumpur"}],
        "PostalZone": [{"_": "50480"}],
        "CountrySubentityCode": [{"_": "10"}],
        "AddressLine": [
            {"Line": [{"_": "Lot 66"}]},
            {"Line": [{"_": "Bangunan Merdeka"}]},
            {"Line": [{"_": "Persiaran Jaya"}]}
        ],
        "Country": [
            {"IdentificationCode": [{"_": "MYS"}]}
        ]
    }
).model_dump(by_alias=True, mode="json")

{'CityName': [{'_': 'Kuala Lumpur'}],
 'PostalZone': [{'_': '50480'}],
 'CountrySubentityCode': [{'_': '10'}],
 'AddressLine': [{'Line': [{'_': 'Lot 66'}]},
  {'Line': [{'_': 'Bangunan Merdeka'}]},
  {'Line': [{'_': 'Persiaran Jaya'}]}],
 'Country': [{'IdentificationCode': [{'_': 'MYS',
     'listID': 'ISO3166-1',
     'listAgencyID': '6'}]}]}

## Source
1. [JSON Invoice Example](https://sdk.myinvois.hasil.gov.my/files/sdksamples/1.0-Invoice-Sample.json)
2. [Invoice 1.0](https://sdk.myinvois.hasil.gov.my/documents/invoice-v1-0/)
3. [API Documentation](https://sdk.myinvois.hasil.gov.my/einvoicingapi/)